ANÁLISIS EXPLORATORIO COMPLETO - AGRICULTURA URBANA VERTICAL CON FOTOLUMINISCENCIA
==================================================================================

Este script realiza un análisis exploratorio exhaustivo de un sistema de agricultura
urbana vertical en ambiente controlado con tecnología de fotoluminiscencia.
Incluye todas las visualizaciones, matrices de correlación y análisis estadísticos.

Dataset: 50,000 registros de sensores en ambiente controlado cerrado
Variables: 26 parámetros ambientales, lumínicos y de control
Tecnología: Sistema fotoluminiscente para optimización del crecimiento

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import pearsonr, spearmanr, chi2_contingency
import warnings
warnings.filterwarnings('ignore')

# Configuración de estilo para visualizaciones
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

Carga y prepara el dataset del sistema de agricultura vertical

In [ ]:
def load_and_prepare_data(file_path):
    """
    Carga y prepara el dataset del sistema de agricultura vertical
    """
    print("SISTEMA DE AGRICULTURA URBANA VERTICAL - ANÁLISIS EXPLORATORIO")
    print("=" * 80)
    print("Cargando datos del ambiente controlado con fotoluminiscencia...")

    df = pd.read_csv(file_path)
    df['timestamp'] = pd.to_datetime(df['timestamp'])

    # Variables adicionales para análisis temporal
    df['fecha'] = df['timestamp'].dt.date
    df['mes'] = df['timestamp'].dt.month
    df['año'] = df['timestamp'].dt.year
    df['semana'] = df['timestamp'].dt.isocalendar().week

    # Mapeo de días de la semana
    dia_nombres = {0: 'Lunes', 1: 'Martes', 2: 'Miércoles', 3: 'Jueves',
                   4: 'Viernes', 5: 'Sábado', 6: 'Domingo'}
    df['dia_nombre'] = df['dia_semana'].map(dia_nombres)

    print(f"Dataset cargado exitosamente:")
    print(f"   Registros: {len(df):,}")
    print(f"   Variables: {df.shape[1]}")
    print(f"   Período: {df['timestamp'].min().date()} a {df['timestamp'].max().date()}")
    print(f"   Ambiente: Controlado cerrado con fotoluminiscencia")

    return df

Crea un dashboard general del sistema

In [ ]:
def create_overview_dashboard(df):
    """
    Crea un dashboard general del sistema
    """
    print("\n" + "=" * 80)
    print("1. DASHBOARD GENERAL DEL SISTEMA")
    print("=" * 80)

    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Dashboard General - Sistema de Agricultura Vertical', fontsize=16, fontweight='bold')

    # 1. Distribución de eficiencia fotosintética
    axes[0,0].hist(df['eficiencia_fotosintetica_pct'], bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[0,0].axvline(df['eficiencia_fotosintetica_pct'].mean(), color='red', linestyle='--',
                     label=f'Media: {df["eficiencia_fotosintetica_pct"].mean():.1f}%')
    axes[0,0].set_title('Distribución Eficiencia Fotosintética')
    axes[0,0].set_xlabel('Eficiencia (%)')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].legend()
    axes[0,0].grid(True, alpha=0.3)

    # 2. Condiciones del ambiente por categoría
    condiciones_counts = df['condiciones_categoria'].value_counts()
    colors = ['#2E8B57', '#32CD32', '#FF6347']
    axes[0,1].pie(condiciones_counts.values, labels=condiciones_counts.index, autopct='%1.1f%%',
                 colors=colors, startangle=90)
    axes[0,1].set_title('Distribución de Condiciones del Ambiente')

    # 3. Calidad del aire
    calidad_counts = df['calidad_aire_categoria'].value_counts()
    axes[0,2].bar(calidad_counts.index, calidad_counts.values,
                 color=['#FF4444', '#FFA500', '#32CD32', '#006400'])
    axes[0,2].set_title('Calidad del Aire en el Sistema')
    axes[0,2].set_xlabel('Categoría')
    axes[0,2].set_ylabel('Frecuencia')
    axes[0,2].tick_params(axis='x', rotation=45)

    # 4. Temperatura vs Humedad por estación
    seasons_colors = {'Primavera': 'green', 'Verano': 'red', 'Otono': 'orange', 'Invierno': 'blue'}
    for season in df['estacion'].unique():
        season_data = df[df['estacion'] == season]
        axes[1,0].scatter(season_data['temperatura_c'], season_data['humedad_rel_pct'],
                         alpha=0.6, label=season, color=seasons_colors[season], s=1)
    axes[1,0].set_title('Temperatura vs Humedad por Estación')
    axes[1,0].set_xlabel('Temperatura (°C)')
    axes[1,0].set_ylabel('Humedad Relativa (%)')
    axes[1,0].legend()

    # 5. Intensidad fotoluminiscente vs Eficiencia
    axes[1,1].scatter(df['fotoluminiscencia_intensidad'], df['eficiencia_fotosintetica_pct'],
                     alpha=0.5, color='purple', s=1)
    axes[1,1].set_title('Fotoluminiscencia vs Eficiencia Fotosintética')
    axes[1,1].set_xlabel('Intensidad Fotoluminiscente')
    axes[1,1].set_ylabel('Eficiencia Fotosintética (%)')

    # Calcular correlación
    corr, p_value = pearsonr(df['fotoluminiscencia_intensidad'], df['eficiencia_fotosintetica_pct'])
    axes[1,1].text(0.05, 0.95, f'r = {corr:.3f}\np = {p_value:.3e}',
                  transform=axes[1,1].transAxes, verticalalignment='top',
                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    # 6. Purificación requerida por calidad del aire
    purif_by_air = df.groupby('calidad_aire_categoria')['purificacion_requerida_pct'].mean()
    axes[1,2].bar(purif_by_air.index, purif_by_air.values, color='skyblue')
    axes[1,2].set_title('Purificación Requerida por Calidad del Aire')
    axes[1,2].set_xlabel('Calidad del Aire')
    axes[1,2].set_ylabel('Purificación Requerida (%)')
    axes[1,2].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

    # Estadísticas básicas del dashboard
    print("\nESTADÍSTICAS CLAVE DEL SISTEMA:")
    print(f"Eficiencia fotosintética promedio: {df['eficiencia_fotosintetica_pct'].mean():.2f}% ± {df['eficiencia_fotosintetica_pct'].std():.2f}%")
    print(f"Temperatura promedio del ambiente: {df['temperatura_c'].mean():.2f}°C ± {df['temperatura_c'].std():.2f}°C")
    print(f"Humedad relativa promedio: {df['humedad_rel_pct'].mean():.2f}% ± {df['humedad_rel_pct'].std():.2f}%")
    print(f"Intensidad fotoluminiscente promedio: {df['fotoluminiscencia_intensidad'].mean():.2f} ± {df['fotoluminiscencia_intensidad'].std():.2f}")
    print(f"Purificación requerida promedio: {df['purificacion_requerida_pct'].mean():.2f}%")


Análisis detallado de condiciones ambientales

In [ ]:
def environmental_conditions_analysis(df):
    """
    Análisis detallado de condiciones ambientales
    """
    print("\n" + "=" * 80)
    print("2. ANÁLISIS DE CONDICIONES AMBIENTALES")
    print("=" * 80)

    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('Análisis de Condiciones Ambientales del Sistema Cerrado', fontsize=16, fontweight='bold')

    # 1. Distribuciones de variables ambientales
    env_vars = ['temperatura_c', 'humedad_rel_pct', 'co2_ppm', 'vpd_kpa']
    colors = ['red', 'blue', 'green', 'orange']

    for i, var in enumerate(env_vars):
        row = i // 2
        col = i % 2

        # Histograma con curva de densidad
        axes[row, col].hist(df[var], bins=50, alpha=0.7, color=colors[i], density=True, edgecolor='black')

        # Curva de densidad
        from scipy.stats import gaussian_kde
        kde = gaussian_kde(df[var].dropna())
        x_range = np.linspace(df[var].min(), df[var].max(), 100)
        axes[row, col].plot(x_range, kde(x_range), 'k-', linewidth=2, label='Densidad')

        # Estadísticas
        mean_val = df[var].mean()
        median_val = df[var].median()
        axes[row, col].axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.2f}')
        axes[row, col].axvline(median_val, color='blue', linestyle='--', label=f'Mediana: {median_val:.2f}')

        axes[row, col].set_title(f'Distribución: {var}')
        axes[row, col].set_xlabel(var)
        axes[row, col].set_ylabel('Densidad')
        axes[row, col].legend()
        axes[row, col].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Análisis de normalidad
    print("\nTEST DE NORMALIDAD (Shapiro-Wilk) - Variables Ambientales:")
    print("-" * 60)
    for var in env_vars:
        # Usar una muestra para el test (máximo 5000 observaciones)
        sample_size = min(5000, len(df[var].dropna()))
        sample = df[var].dropna().sample(sample_size, random_state=42)
        stat, p_value = stats.shapiro(sample)
        print(f"{var}: estadístico = {stat:.4f}, p-valor = {p_value:.2e}")
        if p_value > 0.05:
            print(f"  -> Distribución NORMAL (p > 0.05)")
        else:
            print(f"  -> Distribución NO NORMAL (p ≤ 0.05)")

Análisis específico del sistema de fotoluminiscencia

In [ ]:
def photoluminescence_analysis(df):
    """
    Análisis específico del sistema de fotoluminiscencia
    """
    print("\n" + "=" * 80)
    print("3. ANÁLISIS DEL SISTEMA DE FOTOLUMINISCENCIA")
    print("=" * 80)

    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Sistema de Fotoluminiscencia - Análisis Lumínico', fontsize=16, fontweight='bold')

    # 1. Distribución del espectro lumínico
    espectro_counts = df['espectro_pico_nm'].value_counts().head(15)
    axes[0,0].bar(espectro_counts.index.astype(str), espectro_counts.values, color='violet')
    axes[0,0].set_title('Distribución del Espectro Lumínico (Top 15)')
    axes[0,0].set_xlabel('Longitud de Onda (nm)')
    axes[0,0].set_ylabel('Frecuencia')
    axes[0,0].tick_params(axis='x', rotation=45)

    # 2. PAR vs PPFD
    axes[0,1].scatter(df['par_umol_m2_s'], df['ppfd_umol_m2_s'], alpha=0.5, color='gold', s=1)
    axes[0,1].set_title('PAR vs PPFD')
    axes[0,1].set_xlabel('PAR (μmol/m²/s)')
    axes[0,1].set_ylabel('PPFD (μmol/m²/s)')

    # Correlación PAR-PPFD
    corr_par_ppfd, _ = pearsonr(df['par_umol_m2_s'], df['ppfd_umol_m2_s'])
    axes[0,1].text(0.05, 0.95, f'r = {corr_par_ppfd:.3f}',
                  transform=axes[0,1].transAxes, verticalalignment='top',
                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    # 3. Intensidad fotoluminiscente por hora del día
    hourly_photolum = df.groupby('hora_dia')['fotoluminiscencia_intensidad'].agg(['mean', 'std'])
    axes[0,2].plot(hourly_photolum.index, hourly_photolum['mean'], 'o-', color='purple', linewidth=2)
    axes[0,2].fill_between(hourly_photolum.index,
                          hourly_photolum['mean'] - hourly_photolum['std'],
                          hourly_photolum['mean'] + hourly_photolum['std'],
                          alpha=0.3, color='purple')
    axes[0,2].set_title('Intensidad Fotoluminiscente por Hora')
    axes[0,2].set_xlabel('Hora del Día')
    axes[0,2].set_ylabel('Intensidad Fotoluminiscente')
    axes[0,2].grid(True, alpha=0.3)

    # 4. Eficiencia fotosintética por estación
    seasonal_efficiency = df.groupby('estacion')['eficiencia_fotosintetica_pct'].agg(['mean', 'std', 'count'])
    x_pos = np.arange(len(seasonal_efficiency.index))
    axes[1,0].bar(x_pos, seasonal_efficiency['mean'], yerr=seasonal_efficiency['std'],
                 capsize=5, color=['green', 'red', 'orange', 'blue'], alpha=0.7)
    axes[1,0].set_title('Eficiencia Fotosintética por Estación')
    axes[1,0].set_xlabel('Estación')
    axes[1,0].set_ylabel('Eficiencia Fotosintética (%)')
    axes[1,0].set_xticks(x_pos)
    axes[1,0].set_xticklabels(seasonal_efficiency.index)

    # 5. Heatmap de eficiencia por hora y día de la semana
    efficiency_pivot = df.pivot_table(values='eficiencia_fotosintetica_pct',
                                    index='dia_nombre', columns='hora_dia', aggfunc='mean')

    # Reordenar días de la semana
    day_order = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    efficiency_pivot = efficiency_pivot.reindex(day_order)

    im = axes[1,1].imshow(efficiency_pivot.values, cmap='RdYlGn', aspect='auto')
    axes[1,1].set_title('Heatmap: Eficiencia por Día y Hora')
    axes[1,1].set_xlabel('Hora del Día')
    axes[1,1].set_ylabel('Día de la Semana')
    axes[1,1].set_xticks(range(0, 24, 4))
    axes[1,1].set_xticklabels(range(0, 24, 4))
    axes[1,1].set_yticks(range(len(day_order)))
    axes[1,1].set_yticklabels(day_order)

    # Colorbar
    cbar = plt.colorbar(im, ax=axes[1,1])
    cbar.set_label('Eficiencia Fotosintética (%)')

    # 6. Relación espectro-eficiencia
    spectral_efficiency = df.groupby('espectro_pico_nm')['eficiencia_fotosintetica_pct'].mean()
    top_spectra = spectral_efficiency.nlargest(10)

    axes[1,2].scatter(df['espectro_pico_nm'], df['eficiencia_fotosintetica_pct'],
                     alpha=0.3, color='indigo', s=1)
    axes[1,2].scatter(top_spectra.index, top_spectra.values,
                     color='red', s=50, alpha=0.8, label='Top 10 eficiencias')
    axes[1,2].set_title('Espectro Lumínico vs Eficiencia')
    axes[1,2].set_xlabel('Longitud de Onda (nm)')
    axes[1,2].set_ylabel('Eficiencia Fotosintética (%)')
    axes[1,2].legend()

    plt.tight_layout()
    plt.show()

    # Estadísticas del sistema fotoluminiscente
    print("\nESTADÍSTICAS DEL SISTEMA DE FOTOLUMINISCENCIA:")
    print("-" * 55)
    print(f"Rango de PAR: {df['par_umol_m2_s'].min():.1f} - {df['par_umol_m2_s'].max():.1f} μmol/m²/s")
    print(f"Rango de PPFD: {df['ppfd_umol_m2_s'].min():.1f} - {df['ppfd_umol_m2_s'].max():.1f} μmol/m²/s")
    print(f"Correlación PAR-PPFD: {corr_par_ppfd:.4f}")
    print(f"Espectro más eficiente: {top_spectra.index[0]} nm ({top_spectra.values[0]:.2f}% eficiencia)")
    print(f"Intensidad fotoluminiscente promedio: {df['fotoluminiscencia_intensidad'].mean():.2f}")

 Análisis de calidad del aire del ambiente controlado

In [ ]:
def air_quality_analysis(df):
    """
    Análisis de calidad del aire del ambiente controlado
    """
    print("\n" + "=" * 80)
    print("4. ANÁLISIS DE CALIDAD DEL AIRE")
    print("=" * 80)

    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Calidad del Aire - Sistema de Ambiente Controlado', fontsize=16, fontweight='bold')

    air_vars = ['pm2_5_ugm3', 'pm10_ugm3', 'vocs_mgm3', 'no2_ugm3', 'o3_ugm3', 'aqi_indice']

    # Boxplots para cada contaminante
    for i, var in enumerate(air_vars):
        row = i // 3
        col = i % 3

        # Boxplot por calidad del aire
        air_categories = df['calidad_aire_categoria'].dropna().unique()
        data_by_category = [df[df['calidad_aire_categoria'] == cat][var].dropna()
                           for cat in air_categories]

        bp = axes[row, col].boxplot(data_by_category, labels=air_categories, patch_artist=True)

        # Colorear boxplots
        colors = ['red', 'orange', 'lightgreen', 'darkgreen']
        for patch, color in zip(bp['boxes'], colors[:len(bp['boxes'])]):
            patch.set_facecolor(color)
            patch.set_alpha(0.7)

        axes[row, col].set_title(f'Distribución: {var}')
        axes[row, col].set_ylabel(var)
        axes[row, col].tick_params(axis='x', rotation=45)
        axes[row, col].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

    # Matriz de correlación de contaminantes
    print("\nMATRIZ DE CORRELACIÓN - CONTAMINANTES:")
    print("-" * 45)

    fig, ax = plt.subplots(figsize=(10, 8))
    corr_matrix = df[air_vars].corr()

    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
                square=True, ax=ax, cbar_kws={"shrink": .8})
    ax.set_title('Matriz de Correlación - Contaminantes del Aire')
    plt.tight_layout()
    plt.show()

    # Análisis de purificación
    print("\nANÁLISIS DEL SISTEMA DE PURIFICACIÓN:")
    print("-" * 40)

    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Purificación por calidad del aire
    purif_by_air = df.groupby('calidad_aire_categoria')['purificacion_requerida_pct'].agg(['mean', 'std', 'count'])
    x_pos = np.arange(len(purif_by_air.index))

    axes[0].bar(x_pos, purif_by_air['mean'], yerr=purif_by_air['std'],
               capsize=5, color=['red', 'orange', 'lightgreen', 'darkgreen'], alpha=0.7)
    axes[0].set_title('Purificación Requerida por Calidad del Aire')
    axes[0].set_xlabel('Calidad del Aire')
    axes[0].set_ylabel('Purificación Requerida (%)')
    axes[0].set_xticks(x_pos)
    axes[0].set_xticklabels(purif_by_air.index)

    # Distribución de purificación requerida
    axes[1].hist(df['purificacion_requerida_pct'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
    axes[1].axvline(df['purificacion_requerida_pct'].mean(), color='red', linestyle='--',
                   label=f'Media: {df["purificacion_requerida_pct"].mean():.1f}%')
    axes[1].set_title('Distribución de Purificación Requerida')
    axes[1].set_xlabel('Purificación Requerida (%)')
    axes[1].set_ylabel('Frecuencia')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()

Análisis exhaustivo de correlaciones entre todas las variables

In [ ]:
def correlation_matrix_analysis(df):
    """
    Análisis exhaustivo de correlaciones entre todas las variables
    """
    print("\n" + "=" * 80)
    print("5. ANÁLISIS DE CORRELACIONES - MATRIZ COMPLETA")
    print("=" * 80)

    # Variables numéricas para análisis
    numeric_vars = df.select_dtypes(include=[np.number]).columns.tolist()
    exclude_vars = ['mes', 'año', 'semana']  # Excluir variables temporales derivadas
    analysis_vars = [var for var in numeric_vars if var not in exclude_vars]

    # Matriz de correlación completa
    corr_matrix = df[analysis_vars].corr()

    # Crear figura grande para matriz completa
    fig, ax = plt.subplots(figsize=(16, 14))

    # Heatmap con dendrograma
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='RdBu_r', center=0,
                square=True, ax=ax, cbar_kws={"shrink": .6}, fmt='.2f',
                annot_kws={'size': 8})
    ax.set_title('Matriz de Correlación Completa - Sistema de Agricultura Vertical', fontsize=14)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

    # Identificar correlaciones más fuertes
    print("\nCORRELACIONES MÁS FUERTES (|r| > 0.5):")
    print("-" * 45)

    strong_correlations = []
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            var1 = corr_matrix.columns[i]
            var2 = corr_matrix.columns[j]
            corr_value = corr_matrix.iloc[i, j]
            if abs(corr_value) > 0.5:
                strong_correlations.append((var1, var2, corr_value))

    strong_correlations.sort(key=lambda x: abs(x[2]), reverse=True)

    for i, (var1, var2, corr) in enumerate(strong_correlations[:15]):
        print(f"{i+1:2d}. {var1} ↔ {var2}: r = {corr:.3f}")

    # Correlaciones específicas con eficiencia fotosintética
    print("\nCORRELACIONES CON EFICIENCIA FOTOSINTÉTICA:")
    print("-" * 50)
    efficiency_corr = corr_matrix['eficiencia_fotosintetica_pct'].sort_values(key=abs, ascending=False)
    for var, corr in efficiency_corr.head(10).items():
        if var != 'eficiencia_fotosintetica_pct':
            print(f"{var}: r = {corr:.3f}")

    # Crear mapa de calor específico para variables más importantes
    key_vars = ['temperatura_c', 'humedad_rel_pct', 'co2_ppm', 'par_umol_m2_s',
                'ppfd_umol_m2_s', 'fotoluminiscencia_intensidad', 'eficiencia_fotosintetica_pct',
                'aqi_indice', 'purificacion_requerida_pct']

    fig, ax = plt.subplots(figsize=(10, 8))
    key_corr = df[key_vars].corr()
    sns.heatmap(key_corr, annot=True, cmap='RdBu_r', center=0, square=True,
                ax=ax, cbar_kws={"shrink": .8}, fmt='.3f')
    ax.set_title('Correlaciones - Variables Clave del Sistema')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.show()

Análisis de patrones temporales en el sistema

In [ ]:
def temporal_patterns_analysis(df):
    """
    Análisis de patrones temporales en el sistema
    """
    print("\n" + "=" * 80)
    print("6. ANÁLISIS DE PATRONES TEMPORALES")
    print("=" * 80)

    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('Patrones Temporales - Sistema de Agricultura Vertical', fontsize=16, fontweight='bold')

    # 1. Eficiencia por hora del día
    hourly_efficiency = df.groupby('hora_dia')['eficiencia_fotosintetica_pct'].agg(['mean', 'std'])
    axes[0,0].plot(hourly_efficiency.index, hourly_efficiency['mean'], 'o-', linewidth=2, markersize=6)
    axes[0,0].fill_between(hourly_efficiency.index,
                          hourly_efficiency['mean'] - hourly_efficiency['std'],
                          hourly_efficiency['mean'] + hourly_efficiency['std'],
                          alpha=0.3)
    axes[0,0].set_title('Eficiencia Fotosintética por Hora del Día')
    axes[0,0].set_xlabel('Hora')
    axes[0,0].set_ylabel('Eficiencia (%)')
    axes[0,0].grid(True, alpha=0.3)
    axes[0,0].set_xticks(range(0, 24, 4))

    # 2. Variables ambientales por día de la semana
    weekly_env = df.groupby('dia_nombre')[['temperatura_c', 'humedad_rel_pct', 'co2_ppm']].mean()
    day_order = ['Lunes', 'Martes', 'Miércoles', 'Jueves', 'Viernes', 'Sábado', 'Domingo']
    weekly_env = weekly_env.reindex(day_order)

    x_pos = np.arange(len(day_order))
    width = 0.25

    axes[0,1].bar(x_pos - width, weekly_env['temperatura_c'], width, label='Temperatura (°C)', alpha=0.8)
    axes[0,1].bar(x_pos, weekly_env['humedad_rel_pct'], width, label='Humedad (%)', alpha=0.8)
    axes[0,1].bar(x_pos + width, weekly_env['co2_ppm']/10, width, label='CO2 (ppm/10)', alpha=0.8)

    axes[0,1].set_title('Variables Ambientales por Día de la Semana')
    axes[0,1].set_xlabel('Día')
    axes[0,1].set_ylabel('Valor')
    axes[0,1].set_xticks(x_pos)
    axes[0,1].set_xticklabels(day_order, rotation=45)
    axes[0,1].legend()
    axes[0,1].grid(True, alpha=0.3)

    # 3. Tendencia semanal de eficiencia
    weekly_efficiency = df.groupby('semana')['eficiencia_fotosintetica_pct'].mean()
    axes[1,0].plot(weekly_efficiency.index, weekly_efficiency.values, 'o-', linewidth=2)
    axes[1,0].set_title('Tendencia Semanal de Eficiencia')
    axes[1,0].set_xlabel('Semana del Año')
    axes[1,0].set_ylabel('Eficiencia Promedio (%)')
    axes[1,0].grid(True, alpha=0.3)

    # Línea de tendencia
    z = np.polyfit(weekly_efficiency.index, weekly_efficiency.values, 1)
    p = np.poly1d(z)
    axes[1,0].plot(weekly_efficiency.index, p(weekly_efficiency.index), "--", alpha=0.8, color='red')

    # 4. Calidad del aire por estación
    seasonal_air = pd.crosstab(df['estacion'], df['calidad_aire_categoria'], normalize='index') * 100
    seasonal_air.plot(kind='bar', stacked=True, ax=axes[1,1],
                     color=['red', 'orange', 'lightgreen', 'darkgreen'])
    axes[1,1].set_title('Distribución de Calidad del Aire por Estación')
    axes[1,1].set_xlabel('Estación')
    axes[1,1].set_ylabel('Porcentaje (%)')
    axes[1,1].tick_params(axis='x', rotation=45)
    axes[1,1].legend(title='Calidad del Aire', bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.show()

    # Estadísticas temporales
    print("\nESTADÍSTICAS DE PATRONES TEMPORALES:")
    print("-" * 45)
    print(f"Hora de máxima eficiencia: {hourly_efficiency['mean'].idxmax()}:00")
    print(f"Eficiencia máxima: {hourly_efficiency['mean'].max():.2f}%")
    print(f"Hora de mínima eficiencia: {hourly_efficiency['mean'].idxmin()}:00")
    print(f"Eficiencia mínima: {hourly_efficiency['mean'].min():.2f}%")
    print(f"Variación diaria promedio: {hourly_efficiency['std'].mean():.2f}%")

Análisis del sistema de control y ajustes automáticos

In [ ]:
def control_system_analysis(df):
    """
    Análisis del sistema de control y ajustes automáticos
    """
    print("\n" + "=" * 80)
    print("7. ANÁLISIS DEL SISTEMA DE CONTROL AUTOMÁTICO")
    print("=" * 80)

    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Sistema de Control Automático - Ajustes y Respuestas', fontsize=16, fontweight='bold')

    adjustment_vars = ['ajuste_temp_c', 'ajuste_humedad_pct', 'ajuste_co2_ppm', 'ajuste_par_umol']
    colors = ['red', 'blue', 'green', 'orange']

    # 1-4. Distribución de ajustes
    for i, var in enumerate(adjustment_vars):
        row = i // 2
        col = i % 2

        # Histograma de ajustes
        axes[row, col].hist(df[var], bins=50, alpha=0.7, color=colors[i], edgecolor='black')
        axes[row, col].axvline(0, color='black', linestyle='-', linewidth=2, label='Sin ajuste')
        axes[row, col].axvline(df[var].mean(), color='red', linestyle='--',
                              label=f'Media: {df[var].mean():.2f}')
        axes[row, col].set_title(f'Distribución de Ajustes: {var}')
        axes[row, col].set_xlabel('Valor del Ajuste')
        axes[row, col].set_ylabel('Frecuencia')
        axes[row, col].legend()
        axes[row, col].grid(True, alpha=0.3)

        # Estadísticas de ajustes
        positive = (df[var] > 0).sum()
        negative = (df[var] < 0).sum()
        zero = (df[var] == 0).sum()
        total = len(df)

        # Añadir texto con estadísticas
        text = f'Pos: {positive/total*100:.1f}%\nNeg: {negative/total*100:.1f}%\nCero: {zero/total*100:.1f}%'
        axes[row, col].text(0.02, 0.98, text, transform=axes[row, col].transAxes,
                           verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    # 5. Relación entre condiciones y ajustes
    adjustment_by_condition = df.groupby('condiciones_categoria')[adjustment_vars].mean()

    x_pos = np.arange(len(adjustment_by_condition.index))
    width = 0.2

    for i, var in enumerate(adjustment_vars):
        axes[1, 2].bar(x_pos + i*width, adjustment_by_condition[var], width,
                      label=var, alpha=0.8, color=colors[i])

    axes[1, 2].set_title('Ajustes Promedio por Condición del Ambiente')
    axes[1, 2].set_xlabel('Condición')
    axes[1, 2].set_ylabel('Ajuste Promedio')
    axes[1, 2].set_xticks(x_pos + width * 1.5)
    axes[1, 2].set_xticklabels(adjustment_by_condition.index)
    axes[1, 2].legend()
    axes[1, 2].grid(True, alpha=0.3)
    axes[1, 2].axhline(y=0, color='black', linestyle='-', alpha=0.5)

    plt.tight_layout()
    plt.show()

    # Análisis de correlación entre ajustes y condiciones
    print("\nCORRELACIONES AJUSTES vs CONDICIONES AMBIENTALES:")
    print("-" * 55)

    base_vars = ['temperatura_c', 'humedad_rel_pct', 'co2_ppm', 'par_umol_m2_s']

    for adj_var, base_var in zip(adjustment_vars, base_vars):
        corr, p_value = pearsonr(df[adj_var], df[base_var])
        print(f"{adj_var} ↔ {base_var}: r = {corr:.3f} (p = {p_value:.3e})")

    # Análisis de efectividad del control
    print("\nEFECTIVIDAD DEL SISTEMA DE CONTROL:")
    print("-" * 40)

    # Calcular si los ajustes mejoran la eficiencia
    df['ajuste_total'] = np.sqrt(df['ajuste_temp_c']**2 + df['ajuste_humedad_pct']**2 +
                                df['ajuste_co2_ppm']**2 + df['ajuste_par_umol']**2)

    # Dividir en cuartiles de ajuste
    df['cuartil_ajuste'] = pd.qcut(df['ajuste_total'], 4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
    efficiency_by_adjustment = df.groupby('cuartil_ajuste')['eficiencia_fotosintetica_pct'].mean()

    print("Eficiencia por nivel de ajuste:")
    for cuartil, efficiency in efficiency_by_adjustment.items():
        print(f"  {cuartil}: {efficiency:.2f}%")

Análisis estadístico avanzado

In [ ]:
def advanced_statistical_analysis(df):
    """
    Análisis estadístico avanzado
    """
    print("\n" + "=" * 80)
    print("8. ANÁLISIS ESTADÍSTICO AVANZADO")
    print("=" * 80)

    # Test ANOVA para diferencias entre grupos
    print("\nTEST ANOVA - Diferencias entre grupos:")
    print("-" * 40)

    # ANOVA para eficiencia por estación
    groups_season = [df[df['estacion'] == season]['eficiencia_fotosintetica_pct'].dropna()
                    for season in df['estacion'].unique()]
    f_stat_season, p_val_season = stats.f_oneway(*groups_season)
    print(f"Eficiencia por Estación: F = {f_stat_season:.3f}, p = {p_val_season:.3e}")

    # ANOVA para eficiencia por condición
    groups_condition = [df[df['condiciones_categoria'] == cond]['eficiencia_fotosintetica_pct'].dropna()
                       for cond in df['condiciones_categoria'].unique()]
    f_stat_cond, p_val_cond = stats.f_oneway(*groups_condition)
    print(f"Eficiencia por Condición: F = {f_stat_cond:.3f}, p = {p_val_cond:.3e}")

    # Test de independencia Chi-cuadrado
    print("\nTEST CHI-CUADRADO - Independencia entre variables categóricas:")
    print("-" * 65)

    # Tabla de contingencia: Estación vs Calidad del aire
    contingency_table = pd.crosstab(df['estacion'], df['calidad_aire_categoria'])
    chi2, p_chi2, dof, expected = chi2_contingency(contingency_table)
    print(f"Estación vs Calidad del Aire: χ² = {chi2:.3f}, p = {p_chi2:.3e}")

    # Análisis de outliers multivariado
    print("\nANÁLISIS DE OUTLIERS MULTIVARIADO:")
    print("-" * 40)

    # Variables para análisis multivariado
    key_vars = ['temperatura_c', 'humedad_rel_pct', 'co2_ppm', 'eficiencia_fotosintetica_pct']

    # Distancia de Mahalanobis
    from scipy.spatial.distance import mahalanobis

    data_subset = df[key_vars].dropna()
    mean = data_subset.mean().values
    cov = data_subset.cov().values

    # Calcular distancias de Mahalanobis
    mahal_distances = []
    for i, row in data_subset.iterrows():
        try:
            dist = mahalanobis(row.values, mean, np.linalg.inv(cov))
            mahal_distances.append(dist)
        except:
            mahal_distances.append(np.nan)

    mahal_distances = np.array(mahal_distances)
    threshold = np.percentile(mahal_distances[~np.isnan(mahal_distances)], 95)
    outliers_mahal = np.sum(mahal_distances > threshold)

    print(f"Outliers multivariados (Mahalanobis): {outliers_mahal} ({outliers_mahal/len(data_subset)*100:.2f}%)")

    # Visualización de outliers
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))

    # Distribución de distancias de Mahalanobis
    axes[0].hist(mahal_distances[~np.isnan(mahal_distances)], bins=50, alpha=0.7, color='skyblue')
    axes[0].axvline(threshold, color='red', linestyle='--', label=f'Umbral (95%): {threshold:.2f}')
    axes[0].set_title('Distribución de Distancias de Mahalanobis')
    axes[0].set_xlabel('Distancia de Mahalanobis')
    axes[0].set_ylabel('Frecuencia')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    # Q-Q plot para normalidad de eficiencia
    from scipy import stats as scipy_stats
    scipy_stats.probplot(df['eficiencia_fotosintetica_pct'].dropna(), dist="norm", plot=axes[1])
    axes[1].set_title('Q-Q Plot - Eficiencia Fotosintética')
    axes[1].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.show()


Insights para optimización del rendimiento del sistema

In [ ]:
def performance_optimization_insights(df):
    """
    Insights para optimización del rendimiento del sistema
    """
    print("\n" + "=" * 80)
    print("9. INSIGHTS PARA OPTIMIZACIÓN DEL RENDIMIENTO")
    print("=" * 80)

    # Identificar condiciones óptimas
    top_efficiency = df.nlargest(int(len(df) * 0.1), 'eficiencia_fotosintetica_pct')  # Top 10%

    print("\nCONDICIONES PARA MÁXIMA EFICIENCIA (Top 10%):")
    print("-" * 50)

    optimal_conditions = {
        'temperatura_c': (top_efficiency['temperatura_c'].mean(), top_efficiency['temperatura_c'].std()),
        'humedad_rel_pct': (top_efficiency['humedad_rel_pct'].mean(), top_efficiency['humedad_rel_pct'].std()),
        'co2_ppm': (top_efficiency['co2_ppm'].mean(), top_efficiency['co2_ppm'].std()),
        'par_umol_m2_s': (top_efficiency['par_umol_m2_s'].mean(), top_efficiency['par_umol_m2_s'].std()),
        'fotoluminiscencia_intensidad': (top_efficiency['fotoluminiscencia_intensidad'].mean(),
                                       top_efficiency['fotoluminiscencia_intensidad'].std())
    }

    for var, (mean_val, std_val) in optimal_conditions.items():
        print(f"{var}: {mean_val:.2f} ± {std_val:.2f}")

    # Análisis de eficiencia por rangos
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Análisis de Rangos Óptimos para Máxima Eficiencia', fontsize=16, fontweight='bold')

    # Variables clave para análisis de rangos
    range_vars = ['temperatura_c', 'humedad_rel_pct', 'co2_ppm', 'fotoluminiscencia_intensidad']

    for i, var in enumerate(range_vars):
        row = i // 2
        col = i % 2

        # Crear bins y calcular eficiencia promedio por bin
        df[f'{var}_bin'] = pd.cut(df[var], bins=20)
        efficiency_by_bin = df.groupby(f'{var}_bin')['eficiencia_fotosintetica_pct'].agg(['mean', 'count'])
        efficiency_by_bin = efficiency_by_bin[efficiency_by_bin['count'] >= 50]  # Solo bins con suficientes datos

        # Obtener centros de los bins
        bin_centers = [interval.mid for interval in efficiency_by_bin.index]

        axes[row, col].plot(bin_centers, efficiency_by_bin['mean'], 'o-', linewidth=2, markersize=6)
        axes[row, col].set_title(f'Eficiencia vs {var}')
        axes[row, col].set_xlabel(var)
        axes[row, col].set_ylabel('Eficiencia Promedio (%)')
        axes[row, col].grid(True, alpha=0.3)

        # Marcar rango óptimo
        optimal_mean, optimal_std = optimal_conditions[var]
        axes[row, col].axvspan(optimal_mean - optimal_std, optimal_mean + optimal_std,
                              alpha=0.3, color='green', label='Rango Óptimo')
        axes[row, col].legend()

    plt.tight_layout()
    plt.show()

    # Análisis de combinaciones óptimas
    print("\nCOMBINACIONES ÓPTIMAS DE VARIABLES:")
    print("-" * 40)

    # Crear grupos de eficiencia
    df['grupo_eficiencia'] = pd.cut(df['eficiencia_fotosintetica_pct'],
                                   bins=[0, 40, 50, 60, 100],
                                   labels=['Baja', 'Media', 'Alta', 'Muy Alta'])

    efficiency_summary = df.groupby('grupo_eficiencia')[
        ['temperatura_c', 'humedad_rel_pct', 'co2_ppm', 'fotoluminiscencia_intensidad']
    ].agg(['mean', 'std']).round(2)

    print("Condiciones promedio por grupo de eficiencia:")
    print(efficiency_summary)

    # Recomendaciones específicas
    print("\nRECOMENDACIONES PARA OPTIMIZACIÓN:")
    print("-" * 40)

    recommendations = []

    # Análisis de correlaciones para recomendaciones
    corr_with_efficiency = df[['temperatura_c', 'humedad_rel_pct', 'co2_ppm',
                              'fotoluminiscencia_intensidad', 'eficiencia_fotosintetica_pct']].corr()['eficiencia_fotosintetica_pct']

    for var in ['temperatura_c', 'humedad_rel_pct', 'co2_ppm', 'fotoluminiscencia_intensidad']:
        corr_value = corr_with_efficiency[var]
        optimal_mean, optimal_std = optimal_conditions[var]
        current_mean = df[var].mean()

        if corr_value > 0.1:
            if current_mean < optimal_mean:
                recommendations.append(f"Incrementar {var} hacia {optimal_mean:.2f}")
            elif current_mean > optimal_mean:
                recommendations.append(f"Reducir {var} hacia {optimal_mean:.2f}")
        elif corr_value < -0.1:
            if current_mean > optimal_mean:
                recommendations.append(f"Reducir {var} hacia {optimal_mean:.2f}")
            elif current_mean < optimal_mean:
                recommendations.append(f"Incrementar {var} hacia {optimal_mean:.2f}")

    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec}")